In [130]:
import pandas as pd 
from sqlalchemy import create_engine
import os

In [131]:
csv_file_1 = os.path.join("..", "Weather Project", "Resources", "beach-lab-data.csv")
csv_file_2 = os.path.join("..", "Weather Project", "Resources","beach-water-quality-automated-sensors.csv")
csv_json = os.path.join("..", "Weather Project", "Resources", "beach-swim-advisories.json")

chicago_beach_dna_and_culture_tests_df = pd.read_csv(csv_file_1)
chicago_beach_parameters_df = pd.read_csv(csv_file_2)
swim_advisory_df = pd.read_json(csv_json)

In [132]:
raw_clean_chicago_beach_dna_and_culture_tests_df = chicago_beach_dna_and_culture_tests_df[["DNA Sample Timestamp", "Beach",	"DNA Sample 1 Reading",	
                                                                                           "DNA Sample 2 Reading", "DNA Reading Mean", "Culture Sample 1 Timestamp",	                                                                                       "Culture Sample 1 Reading", "Culture Sample 2 Reading", 
                                                                                           "Culture Reading Mean", "Latitude", "Longitude", "Location"]]
raw_clean_swim_advisory_df = swim_advisory_df[["Beach Name", "Date", "Predicted Level", "Probability", "Swim Advisory"]]
raw_chicago_beach_parameters_df = chicago_beach_parameters_df[["Beach Name", "Measurement Timestamp", "Water Temperature", "Turbidity", "Transducer Depth", 
                                                               "Wave Height", "Wave Period", "Battery Life"]]

In [133]:
clean_chicago_beach_dna_and_culture_tests_df = raw_clean_chicago_beach_dna_and_culture_tests_df.rename(columns={"DNA Sample Timestamp":                                                                                                                                      "DNA_Sample_Timestamp",                      
                                                                                                                "DNA Sample 1 Reading":                 
                                                                                                                "DNA_Sample_1_Reading",
                                                                                                                "DNA Sample 2 Reading":                                                                                                                                      "DNA_Sample_2_Reading",
                                                                                                                "DNA Reading Mean": 
                                                                                                                "DNA_Reading_Mean",
                                                                                                                "Culture Sample 1 Timestamp": 
                                                                                                                "Culture_Sample_1_Timestamp",     
                                                                                                                "Culture Sample 1 Reading":                                                                                                                                  "Culture_Sample_1_Reading",
                                                                                                                "Culture Sample 2 Reading":             
                                                                                                                "Culture_Sample_2_Reading",
                                                                                                                "Culture Reading Mean":                 
                                                                                                                "Culture_Reading_Mean"})

clean_swim_advisory_df = raw_clean_swim_advisory_df.rename(columns={"Beach Name": "Beach", "Predicted Level": "Predicted_Level",                                                                                                 "Swim Advisory": "Swim_Advisory"})

clean_chicago_beach_parameters_df = raw_chicago_beach_parameters_df.rename(columns={"Beach Name": "Beach_Name", "Measurement Timestamp":                                                                                                         "Measurement_Timestamp",                                                                                                                                     "Water Temperature": "Water_Temperature", 
                                                                                    "Transducer Depth": "Transducer_Depth",                                                                                                                      "Wave Height": "Wave_Height", 
                                                                                    "Wave Period": "Wave_Period", 
                                                                                    "Battery Life": "Battery_Life"})

In [134]:
sql_lite = "sqlite:///my_sqlite.db"
engine = create_engine(sql_lite)

In [135]:
clean_chicago_beach_dna_and_culture_tests_df.to_sql(name="clean_chicago_beach_dna_and_culture_tests_df", con=engine, if_exists="replace", index=False)
clean_swim_advisory_df.to_sql(name="clean_swim_advisory_df", con=engine, if_exists="replace", index=False)
clean_chicago_beach_parameters_df.to_sql(name="clean_chicago_beach_parameters_df", con=engine, if_exists="replace", index=False)

In [136]:
total_beaches_df = pd.read_sql_query("""SELECT Beach,
                                        avg(DNA_Sample_1_Reading) AS DNA_Sample_1_Reading_Mean,                             
                                        avg(DNA_Sample_2_Reading) AS DNA_Sample_2_Reading_Mean, 
                                        avg(DNA_Reading_Mean) AS DNA_Overall_Reading_Mean,  
                                        avg(Culture_Sample_1_Reading) AS Culture_Sample_1_Reading_Mean,                
                                        avg(Culture_Sample_2_Reading) AS Culture_Sample_2_Reading_Mean, 
                                        avg(Culture_Reading_Mean) AS Culture_Overall_Reading_Mean
                                        FROM clean_chicago_beach_dna_and_culture_tests_df  
                                        WHERE Beach <> 'None'
                                        GROUP BY Beach
                                     """, con=engine)

total_beaches_df.to_sql(name="total_beaches_df", con=engine, if_exists="replace", index=False) 

In [137]:
DNA_safe_beaches_df = pd.read_sql_query("""SELECT * FROM total_beaches_df
                                           WHERE (DNA_Sample_1_Reading_Mean < 1000 
                                           AND DNA_Sample_2_Reading_Mean < 1000
                                           AND DNA_Overall_Reading_Mean < 1000) 
                                           OR (DNA_Sample_1_Reading_Mean IS NULL 
                                           OR DNA_Sample_2_Reading_Mean IS NULL                    
                                           OR DNA_Overall_Reading_Mean IS NULL)            
                                        """, con=engine)

DNA_safe_beaches_df.to_sql(name="DNA_safe_beaches_df", con=engine, if_exists="replace", index=False)                                        

DNA_and_culture_safe_beaches_df = pd.read_sql_query("""SELECT * FROM DNA_safe_beaches_df
                                                       WHERE (Culture_Sample_1_Reading_Mean < 235 
                                                       AND Culture_Sample_2_Reading_Mean < 235
                                                       AND Culture_Overall_Reading_Mean < 235) 
                                                       OR (Culture_Sample_1_Reading_Mean IS NULL 
                                                       OR Culture_Sample_2_Reading_Mean IS NULL                    
                                                       OR Culture_Overall_Reading_Mean IS NULL);
                                                    """, con=engine)                                       
 
DNA_and_culture_safe_beaches_df.to_sql(name="DNA_and_culture_safe_beaches_df", con=engine, if_exists="replace", index=False) 

DNA_and_culture_safe_beaches_df

,Beach,DNA_Sample_1_Reading_Mean,DNA_Sample_2_Reading_Mean,DNA_Overall_Reading_Mean,Culture_Sample_1_Reading_Mean,Culture_Sample_2_Reading_Mean,Culture_Overall_Reading_Mean
0,12th Street,477.594118,277.323529,305.977647,164.822777,160.685488,150.768044
1,57th Street,407.446602,345.194175,325.009709,163.935810,169.782805,153.530182
2,Columbia,NaN,NaN,NaN,105.620775,132.175540,98.190493
3,Fargo,NaN,NaN,NaN,55.888776,42.948958,44.014286
4,Foster,256.786408,336.077670,204.372816,144.617466,139.474539,135.405767
5,Hartigan,33.000000,20.000000,25.700000,86.960430,83.922535,79.236237
6,Hartigan (Albion),461.184466,345.088235,352.887379,NaN,NaN,NaN
7,Howard,298.105769,561.764706,327.623077,65.057687,68.979438,63.231566
8,Juneway,NaN,NaN,NaN,68.680127,71.963299,63.567893
9,Lane,NaN,NaN,NaN,146.537809,151.169751,138.361888


In [138]:
unsafe_beaches_df = pd.read_sql_query("""SELECT * FROM total_beaches_df
                                         WHERE DNA_Sample_1_Reading_Mean >= 1000
                                         OR DNA_Sample_2_Reading_Mean >= 1000
                                         OR DNA_Overall_Reading_Mean >= 1000 
                                         OR Culture_Sample_1_Reading_Mean >= 235 
                                         OR Culture_Sample_2_Reading_Mean >= 235
                                         OR Culture_Overall_Reading_Mean >= 235 
                                      """, con=engine)                                       
 
unsafe_beaches_df.to_sql(name="unsafe_beaches_df", con=engine, if_exists="replace", index=False) 

unsafe_beaches_df

,Beach,DNA_Sample_1_Reading_Mean,DNA_Sample_2_Reading_Mean,DNA_Overall_Reading_Mean,Culture_Sample_1_Reading_Mean,Culture_Sample_2_Reading_Mean,Culture_Overall_Reading_Mean
0,63rd Street,863.750893,673.273303,693.698661,327.079103,319.662722,304.352000
1,Calumet,419.256388,426.590308,371.786784,253.946719,241.966629,225.068243
2,Humboldt,3576.528846,3998.471154,3581.267308,35.270000,22.872500,26.770000
3,Margaret T Burroughs,372.342857,395.257143,338.567143,226.162967,245.076988,219.259064
4,Montrose,908.200881,908.769912,702.403084,305.017572,317.136647,294.973933
5,Montrose Dog,NaN,NaN,NaN,328.082460,336.712073,306.591364
6,Rainbow,488.526872,724.042478,477.889868,273.560166,270.017836,252.946327
7,South Shore,434.175893,359.740625,356.683482,243.380414,210.870195,203.557767


In [139]:
safe_beaches_count = len(DNA_and_culture_safe_beaches_df["Beach"])
unsafe_beaches_count = len(unsafe_beaches_df["Beach"])
total_beaches_count = len(total_beaches_df["Beach"])
safe_beaches_percentage = ('{:.2f}%').format((safe_beaches_count/total_beaches_count)*100)
unsafe_beaches_percentage = ('{:.2f}%').format((unsafe_beaches_count/total_beaches_count)*100)
print(f"From the two tables that we previously drew, we can consider that around {safe_beaches_percentage} of the beaches of the city of Chicago are safe.")
print(f"On the other hand, from the two tables that we previously drew, we can consider that around {unsafe_beaches_percentage} of the beaches of the city of Chicago are unsafe.")

From the two tables that we previously drew, we can consider that around 73.33% of the beaches of the city of Chicago are safe.
On the other hand, from the two tables that we previously drew, we can consider that around 26.67% of the beaches of the city of Chicago are unsafe.


In [140]:
swim_advisory_beaches_df = pd.read_sql_query("""SELECT Beach, Probability, 
                                                avg(Predicted_Level) AS Average_Predicted_Level,
                                                Swim_Advisory
                                                FROM clean_swim_advisory_df
                                                WHERE Probability >= 0
                                                GROUP BY Beach
                                                ORDER BY Average_Predicted_Level
                                             """, con=engine) 

swim_advisory_beaches_df.to_sql(name="swim_advisory_beaches_df", con=engine, if_exists="replace", index=False)                                             

swim_advisory_beaches_df

,Beach,Probability,Average_Predicted_Level,Swim_Advisory
0,OakStreet,0.018,24.699020,N
1,Ohio,0.082,27.430392,N
2,Leone,0.006,47.957843,N
3,Foster,0.021,67.088235,N
4,Calumet,0.107,85.576471,N
5,Osterman,0.004,87.411765,N
6,63rdStreet,0.106,98.012745,N
7,Rainbow,0.078,100.073529,N
8,Montrose,0.277,213.367647,N


In [141]:
least_safe_beaches_df = pd.read_sql_query("""SELECT Beach, Probability, 
                                        avg(Predicted_Level) AS Average_Predicted_Level,
                                        Swim_Advisory
                                        FROM clean_swim_advisory_df
                                        WHERE Probability >= 0
                                        AND Predicted_Level >= 235
                                        GROUP BY Beach
                                        ORDER BY Beach
                                        """, con=engine) 

least_safe_beaches_df.to_sql(name="least_safe_beaches_df", con=engine, if_exists="replace", index=False)                                             

least_safe_beaches_df

,Beach,Probability,Average_Predicted_Level,Swim_Advisory
0,63rdStreet,0.621,667.260000,Y
1,Calumet,0.506,269.683333,Y
2,Foster,0.549,309.050000,Y
3,Leone,0.587,311.400000,Y
4,Montrose,0.661,532.222222,Y
5,Osterman,0.558,371.125000,Y
6,Rainbow,0.535,286.100000,Y


In [160]:
safest_beaches_list = list(set(swim_advisory_beaches_df["Beach"]) - set(least_safe_beaches_df["Beach"]))
print(f"The beaches that seem to have their predicted CFU level per 100 ml the lowest throughout the whole year of 2016 are the {safest_beaches_list[0]} and {safest_beaches_list[1]} beaches. Thus, they might be the safest beaches to go to.")

The beaches that seem to have their predicted CFU level per 100 ml the lowest throughout the whole year of 2016 are the OakStreet and Ohio beaches. Thus, they might be the safest beaches to go to.


In [165]:
first_formatter = unsafe_beaches_df
first_formatter["Swim Advisory"] = "N"
second_formatter = DNA_and_culture_safe_beaches_df
second_formatter["Swim Advisory"] = "Y"
raw_total_beaches_with_swim_advisory_df = pd.concat([first_formatter, second_formatter])
total_beaches_with_swim_advisory_df = raw_total_beaches_with_swim_advisory_df.reset_index()
del total_beaches_with_swim_advisory_df["index"]

total_beaches_with_swim_advisory_df.to_sql(name="total_beaches_with_swim_advisory_df", con=engine, if_exists="replace", index=False)     

print("This data represents the DNA and culture tests results of all the beaches of our dataset with a swim advisory given to the beaches that exceed the maximum level for any of the DNA or culture tests. If a swim advisory is given to a beach, the beach gets 'Y' on its part of the 'Swim Advisory' column; otherwise, it gets a 'N'.")
total_beaches_with_swim_advisory_df

This data represents the DNA and culture tests results of all the beaches of our dataset with a swim advisory given to the beaches that exceed the maximum level for any of the DNA or culture tests. If a swim advisory is given to a beach, the beach gets 'Y' on its part of the 'Swim Advisory' column; otherwise, it gets a 'N'.


,Beach,DNA_Sample_1_Reading_Mean,DNA_Sample_2_Reading_Mean,DNA_Overall_Reading_Mean,Culture_Sample_1_Reading_Mean,Culture_Sample_2_Reading_Mean,Culture_Overall_Reading_Mean,Swim Advisory
0,63rd Street,863.750893,673.273303,693.698661,327.079103,319.662722,304.352000,N
1,Calumet,419.256388,426.590308,371.786784,253.946719,241.966629,225.068243,N
2,Humboldt,3576.528846,3998.471154,3581.267308,35.270000,22.872500,26.770000,N
3,Margaret T Burroughs,372.342857,395.257143,338.567143,226.162967,245.076988,219.259064,N
4,Montrose,908.200881,908.769912,702.403084,305.017572,317.136647,294.973933,N
5,Montrose Dog,NaN,NaN,NaN,328.082460,336.712073,306.591364,N
6,Rainbow,488.526872,724.042478,477.889868,273.560166,270.017836,252.946327,N
7,South Shore,434.175893,359.740625,356.683482,243.380414,210.870195,203.557767,N
8,12th Street,477.594118,277.323529,305.977647,164.822777,160.685488,150.768044,Y
9,57th Street,407.446602,345.194175,325.009709,163.935810,169.782805,153.530182,Y


In [166]:
engine.table_names()

['DNA_and_culture_safe_beaches_df',
 'DNA_safe_beaches_df',
 'clean_chicago_beach_dna_and_culture_tests_df',
 'clean_chicago_beach_parameters_df',
 'clean_swim_advisory_df',
 'least_safe_beaches_df',
 'swim_advisory_beaches_df',
 'total_beaches_df',
 'total_beaches_with_swim_advisory_df',
 'unsafe_beaches_df']